<h1> Задание 1 </h1>

In [5]:
import nltk

Зададим правила для анализа предложений: <br /><br />
    1) Вася читает мою книгу <br />
    2) Напиши какое-нибудь письмо <br />
    3) Этот веселый мальчик идет <br />
    4) Он любит читать всякие книги.


In [7]:
rules = """
    S -> NP VP | VP
    NP -> Det Adj N | Det N | NN
    VP -> V VP | V NP | V
    Det -> 'мою' | 'какое-нибудь' | 'этот' | 'всякие'
    Adj -> 'весёлый'
    N -> 'мальчик' | 'книгу' | 'письмо' | 'книги'
    NN -> 'Вася'
    V -> 'читает' | 'Напиши' | 'идёт' | 'любит' | 'читать'
"""

Опишем анализ предложения при помощи Алгоритма CYK:

<table>
    <tr><td>S</td><td></td><td></td><td></td></tr>
    <tr><td></td><td>VP</td><td></td><td></td></tr>
    <tr><td>NP</td><td></td><td>NP</td><td></td></tr>
    <tr><td>NN</td><td>V</td><td>Det</td><td>N</td></tr>
    <tr><td>Вася</td><td>читает</td><td>мою</td><td>книгу</td></tr>
</table>

Сработавшие правила: <br />
NN -> 'Вася' <br />
V -> 'читает' <br />
Det -> 'мою' <br />
N -> 'книгу' <br />
NP -> NN <br />
NP -> Det N <br />
VP -> V NP <br />
S -> NP VP <br />

Также опишем его анализ при помощи Алгоритма Эрли:

0. <br />
$\gamma$ -> . S [0,0] Dummy start state <br />
S -> . NP VP [0,0] Predictor <br />
NP -> . Det Adj N [0,0] Predictor <br />
NP -> . Det N [0,0] Predictor <br />
NP -> . NN [0,0] Predictor <br />
S -> .VP [0,0] Predictor <br />
VP -> .V VP [0,0] Predictor <br />
VP -> . V NP [0,0] Predictor <br />
VP -> . V [0,0] Predictor <br />

1. <br />
NN -> Вася . [0,1] Scanner <br />
NP -> NN . [0,1] Completer <br />
S -> NP . VP [0,1] Completer <br />
VP -> .V VP [1,1] Predictor <br />
VP -> . V NP [1,1] Predictor <br />
VP -> . V [1,1] Predictor <br />

2. <br />
V -> Читает . [1,2] Scanner <br />
VP -> V . [1,2] Completer <br />
VP -> V. VP [1, 2] Completer <br />
VP -> V . NP [2,2] Completer <br />
VP -> . V [2,2] Predictor <br />
VP -> . V VP [2,2] Predictor <br />
VP -> . V NP [2,2] Predictor <br />
NP -> . Det Adj N [2,2] Predictor <br />
NP -> . Det N [2,2] Predictor <br />
NP -> . NN [2,2] Predictor <br />

3. <br />
Det -> мою . [2,3] Scanner <br />
NP -> Det . N [2,3] Completer <br />
NP -> Det . Adj N [2,3] Completer <br />

4. <br />
N -> книгу . [3,4] Scanner <br />
NP -> Det N . [2,4] Completer <br />
VP -> V NP . [1,4] Completer <br />
S -> NP VP . [0,4] Completer <br />

<h1>Задание 2</h2>

In [9]:
grammar = nltk.CFG.fromstring(rules)
parser = nltk.EarleyChartParser(grammar, trace=1)
for tree in parser.parse("Вася читает мою книгу".split()):
    print(tree)

|.   Вася  .  читает .   мою   .  книгу  .|
|[---------]         .         .         .| [0:1] 'Вася'
|.         [---------]         .         .| [1:2] 'читает'
|.         .         [---------]         .| [2:3] 'мою'
|.         .         .         [---------]| [3:4] 'книгу'
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] S  -> * VP
|>         .         .         .         .| [0:0] VP -> * V VP
|>         .         .         .         .| [0:0] VP -> * V NP
|>         .         .         .         .| [0:0] VP -> * V
|>         .         .         .         .| [0:0] NP -> * Det Adj N
|>         .         .         .         .| [0:0] NP -> * Det N
|>         .         .         .         .| [0:0] NP -> * NN
|>         .         .         .         .| [0:0] NN -> * 'Вася'
|[---------]         .         .         .| [0:1] NN -> 'Вася' *
|[---------]         .         .         .| [0:1] NP -> NN *
|[--------->         .         

<h1> Задание 3. </h1>

Для того, чтобы улучшить работу этого парсера с новыми словами будем использовать морфологический тэггер, определять часть речи и создавать новое правила перехода определённой части речи в новое слово:

In [18]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

In [25]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\k1l77\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [63]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\k1l77\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [82]:
tagger = lambda x: pos_tag(x, lang="rus")

In [83]:
def enhanced_parse(textgrammar, sent, tagger, tokenizer):
    textgrammar = textgrammar.splitlines()
    terminals = [rule.split('->')[1].strip(" '") for rule in textgrammar if "-> '" in rule]
    sent = tagger(tokenizer(sent))
    for token, tag in sent:
        if token not in terminals:
            textgrammar.append("    "+tag+" -> '"+token+"'")
    textgrammar = '\n'.join(textgrammar)
    print(textgrammar)
    textgrammar = nltk.CFG.fromstring(textgrammar)
    p = nltk.EarleyChartParser(textgrammar, trace=1)
    for tree in p.parse([i[0] for i in sent]):
        print(tree)

In [84]:
enhanced_parse(rules, "Вася бьёт Петю", tagger, word_tokenize)


    S -> NP VP | VP
    NP -> Det Adj N | Det N | NN
    VP -> V VP | V NP | V
    Det -> 'мою' | 'какое-нибудь' | 'этот' | 'всякие'
    Adj -> 'весёлый'
    N -> 'мальчик' | 'книгу' | 'письмо' | 'книги'
    NN -> 'Вася'
    V -> 'читает' | 'Напиши' | 'идёт' | 'любит' | 'читать'
    V -> 'бьёт'
    S -> 'Петю'
|.    Вася   .    бьёт   .    Петю   .|
|[-----------]           .           .| [0:1] 'Вася'
|.           [-----------]           .| [1:2] 'бьёт'
|.           .           [-----------]| [2:3] 'Петю'
|>           .           .           .| [0:0] S  -> * NP VP
|>           .           .           .| [0:0] S  -> * VP
|>           .           .           .| [0:0] S  -> * 'Петю'
|>           .           .           .| [0:0] VP -> * V VP
|>           .           .           .| [0:0] VP -> * V NP
|>           .           .           .| [0:0] VP -> * V
|>           .           .           .| [0:0] NP -> * Det Adj N
|>           .           .           .| [0:0] NP -> * Det N
|>          

Парсер не срабатывает правильно из-за неправильной работы POS-теггера!